In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
X = []
Y = []
total_class = 43
cur_directory = os.getcwd()

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [ ]:
for index in range(total_class):
    path = os.path.join(cur_directory,'train',str(index))
    images = os.listdir(path)
    for img in images:
        try:
            image = Image.open(path + '\\'+ img)
            image = image.resize((30,30))
            image = np.array(image)
            X.append(image)
            Y.append(index)
        except:
            print('Error loading image')

In [ ]:
X = np.array(X)
Y = np.array(Y)
print(X.shape, Y.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print('Shape of x_train: ', x_train.shape, ' and y_train:',y_train.shape)
print('Shape of x_test: ', x_test.shape, ' and y_test:',y_test.shape)
#one hot encoding the labels
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=x_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 15
history = model.fit(x_train, y_train, batch_size=64, epochs=epochs,validation_data=(x_test, y_test))
model.save('traffic_recognition.h5')

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

In [ ]:
from sklearn.metrics import accuracy_score
import pandas as pd
y_test = pd.read_csv('Test.csv')
labels = y_test['ClassId'].values
img_paths = y_test['Path'].values
test_data=[]
for path in img_paths:
    image = Image.open(path)
    image = image.resize((30,30))
    test_data.append(np.array(image))
test_data = np.array(test_data)
pred = model.predict_classes(test_data)
from sklearn.metrics import accuracy_score
accuracy_score(labels, pred)

In [1]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy
#load the trained model to classify sign
from keras.models import load_model
model = load_model('traffic_recognition.h5')
#dictionary to label all traffic signs class
classes = { 1:'Speed limit (20km/h)',
    2:'Speed limit (30km/h)',
    3:'Speed limit (50km/h)',
    4:'Speed limit (60km/h)',
    5:'Speed limit (70km/h)',
    6:'Speed limit (80km/h)',
    7:'End of speed limit (80km/h)',
    8:'Speed limit (100km/h)',
    9:'Speed limit (120km/h)',
    10:'No passing',
    11:'No passing veh over 3.5 tons',
    12:'Right-of-way at intersection',
    13:'Priority road',
    14:'Yield',
    15:'Stop',
    16:'No vehicles',
    17:'Veh > 3.5 tons prohibited',
    18:'No entry',
    19:'General caution',
    20:'Dangerous curve left',
    21:'Dangerous curve right',
    22:'Double curve',
    23:'Bumpy road',
    24:'Slippery road',
    25:'Road narrows on the right',
    26:'Road work',
    27:'Traffic signals',
    28:'Pedestrians',
    29:'Children crossing',
    30:'Bicycles crossing',
    31:'Beware of ice/snow',
    32:'Wild animals crossing',
    33:'End speed + passing limits',
    34:'Turn right ahead',
    35:'Turn left ahead',
    36:'Ahead only',
    37:'Go straight or right',
    38:'Go straight or left',
    39:'Keep right',
    40:'Keep left',
    41:'Roundabout mandatory',
    42:'End of no passing',
    43:'End no passing veh > 3.5 tons' }

In [2]:
def classify(file_path):
    image = Image.open(file_path)
    image = image.resize((30,30))
    image = numpy.expand_dims(image, axis=0)
    image = numpy.array(image)
    pred = model.predict_classes([image])[0]
    sign = classes[pred+1]
    print(sign)
    result.configure(text=sign)

In [3]:
def show_classify_btn(file_path):
    classify_b=Button(top,text='Classify Image',command=lambda: classify(file_path),padx=10,pady=5)
    classify_b.configure(bg='#364156', fg='white',font=('arial',10,'bold'))
    classify_b.place(relx=0.79,rely=0.46)

In [4]:
def upload_image():
  try:
    file_path=filedialog.askopenfilename()
    uploaded=Image.open(file_path)
    uploaded.thumbnail(((top.winfo_width()/2.25),(top.winfo_height()/2.25)))
    im=ImageTk.PhotoImage(uploaded)
    sign_image.configure(image=im)
    sign_image.image=im
    result.configure(text='')
    show_classify_btn(file_path)
  except:
    pass

In [5]:
if __name__=='__main__':
  #initialise GUI
  top=tk.Tk()
  top.geometry('800x600')
  top.title('Traffic sign recognition')
  top.configure(bg='#f9f6f7')
  heading = Label(top, text='Traffic sign recognition',pady=20, font=('arial',20,'bold'))
  heading.configure(background='#f9f6f7',fg='#364156')
  heading.pack()
  result=Label(top, font=('arial',15,'bold'))
  result.configure(fg='#011638',bg='#f9f6f7')
  sign_image = Label(top)
  upload=Button(top,text='Upload an image',command=upload_image,padx=10,pady=5)
  upload.configure(background='#364156', fg='white',font=('arial',10,'bold'))
  upload.pack(side=BOTTOM,pady=50)
  sign_image.pack(side=BOTTOM,expand=True)
  result.pack(side=BOTTOM,expand=True)
  top.mainloop()

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Speed limit (30km/h)
